In [1]:
import xarray as xr
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import warnings

In [2]:
path='/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Sea level pressure/*.nc'
data=xr.open_mfdataset(path)


In [3]:
data

<xarray.Dataset>
Dimensions:    (lat: 73, lon: 144, time: 15340, nbnds: 2)
Coordinates:
  * lat        (lat) float32 90.0 87.5 85.0 82.5 ... -82.5 -85.0 -87.5 -90.0
  * lon        (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
Dimensions without coordinates: nbnds
Data variables:
    slp        (time, lat, lon) float32 dask.array<chunksize=(365, 73, 144), meta=np.ndarray>
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(365, 2), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis (1981)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/02/06 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [4]:
# data.lat

The data is already in desired form

# Extracting data

In [5]:
lat_range = slice(29,18.0)  # take lat values in backwards directions.
lon_range = slice(85, 97)  # 88.0 degrees east to 92.7 degrees east

# Use the sel method to select data within the specified Bangladesh region
bangladesh_slp = data.sel(lat=lat_range,lon=lon_range)

In [6]:
import pandas as pd
import xarray as xr
from tqdm.auto import tqdm# Import tqdm
warnings.filterwarnings('ignore')

# # Open your netCDF dataset
# ds = xr.open_dataset('your_dataset.nc')

# Define the chunk size
chunk_size = 500  # Adjust this based on your available memory

# Initialize an empty DataFrame to store the results
slp_df = pd.DataFrame(columns=['time', 'lat', 'lon', 'slp'])

# Calculate the number of chunks
num_chunks = len(bangladesh_slp.time)

# Create a tqdm progress bar
for start in tqdm(range(0, num_chunks, chunk_size),desc='Main Loop'):
    end = start + chunk_size
    chunk = bangladesh_slp.isel(time=slice(start, end))

    # Extract the data
    time = chunk.time.values
    lat = chunk.lat.values
    lon = chunk.lon.values
    slp = chunk.slp.values

    # Create a DataFrame for this chunk and append it to the main DataFrame
    chunk_df = pd.DataFrame({
        'time': time.repeat(len(lat) * len(lon)),
        'lat': np.tile(lat, len(time) * len(lon)),
        'lon': np.tile(lon, len(time) * len(lat)),
        'slp': slp.ravel()
    })

    slp_df = slp_df.append(chunk_df, ignore_index=True)

# t2m_df now contains the concatenated data
slp_df

Main Loop:   0%|          | 0/31 [00:00<?, ?it/s]

,time,lat,lon,slp
0,1981-01-01,27.5,85.0,101135.0
1,1981-01-01,25.0,87.5,101170.0
2,1981-01-01,22.5,90.0,101365.0
3,1981-01-01,20.0,92.5,101635.0
4,1981-01-01,27.5,95.0,101815.0
...,...,...,...,...
306795,2022-12-31,20.0,85.0,101760.0
306796,2022-12-31,27.5,87.5,101815.0
306797,2022-12-31,25.0,90.0,101847.5
306798,2022-12-31,22.5,92.5,101787.5
